In [1]:
!pip install 'transformers==4.23.1' torch sentencepiece datasets evaluate sacrebleu scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling

In [2]:
# %cd "/content/drive/MyDrive/rotten-tomatoes"

In [35]:
from datasets import load_dataset
import torch
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from sklearn.metrics import accuracy_score

In [3]:
!mkdir -p data
!python prepare_tomatoes.py

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.
100% 3/3 [00:00<00:00, 262.71it/s]
Saving into: data/train.json
Saving into: data/s2s-train.json
Saving into: data/validation.json
Saving into: data/s2s-validation.json
Saving into: data/test.json
Saving into: data/s2s-test.json


In [4]:
!python run_translation.py \
  --cache_dir .cache_training \
  --model_name_or_path "google/t5-v1_1-base" \
  --train_file data/s2s-train.json \
  --validation_file data/s2s-validation.json \
  --test_file data/s2s-test.json \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --source_lang "text" \
  --target_lang "label" \
  --source_prefix "tomatoes classification" \
  --max_source_length 256 \
  --max_target_length 128 \
  --generation_max_length 128 \
  --do_train \
  --do_eval \
  --do_predict \
  --predict_with_generate \
  --max_eval_samples 2000 \
  --num_train_epochs 5 \
  --save_strategy steps \
  --save_steps 250 \
  --save_total_limit 5 \
  --logging_strategy steps \
  --logging_steps 100 \
  --eval_steps 250 \
  --evaluation_strategy steps \
  --metric_for_best_model accuracy \
  --greater_is_better True \
  --load_best_model_at_end True \
  --output_dir out/tomatoes/t5_v1_1_freeze_base

2023-02-14 21:05:45.251582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 21:05:46.657296: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 21:05:46.657410: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 21:05:46.657430: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [8]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.4 MB/s eta 0:00:00


In [13]:
path = '/content/out/tomatoes/t5_v1_1_freeze_base'
model = AutoModelForSeq2SeqLM.from_pretrained(path, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained(path)
model_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

In [14]:
text = 'This movie sucks!'
text_p = model_pipeline(text, max_length=60)
text_p

[{'generated_text': 'negative'}]

In [15]:
text = "Wow, it's amazing."
text_p = model_pipeline(text, max_length=60)
text_p

[{'generated_text': 'positive'}]

In [29]:
static_que = ' Is this a positive or negative review?'

def eval(dataset):
  result = []

  for text in tqdm(dataset):
    text_q = text + static_que
    prediction = model_pipeline(text_q, max_length=60)
    result.append(prediction[0]['generated_text'])
  return result

In [18]:
dataset = load_dataset('rotten_tomatoes')

  0%|          | 0/3 [00:00<?, ?it/s]

In [50]:
predictions = eval(dataset['test']['text'])

100%|██████████| 1066/1066 [06:41<00:00,  2.65it/s]


In [51]:
label2id = {
    'negative': 0,
    'positive': 1
}

predictions = [label2id[prediction] for prediction in predictions]

In [53]:
accuracy_score(dataset['test']['label'], predictions)

0.6557223264540337